In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Project/2023/BIGCON/

/content/drive/MyDrive/Project/2023/BIGCON


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score

In [5]:
data = pd.read_csv('raw_data.csv', encoding = 'cp949')
data.head()

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,train,전체점포수,프랜차이즈점포수,일반점포수,길단위유동인구,...,생활물가지수,부동산거래대비유동인구,공실률대비매매가임대료,젠트리피케이션,지하철개수,스타벅스개수,대학교여부,65세이상,65세이상_남,65세이상_여
0,202201,1111010100100030100,11110101,A03,4.0,1,85.84,1.45,84.39,14929.0,...,104.55,19.76,73.540769,51.190526,0.0,0.0,0,2144.0,875.0,1269.0
1,202201,1156011100104460000,11560111,A01,5.0,1,295.00,23.00,272.00,92745.0,...,104.55,51.69,68.320000,43.180000,2.0,0.0,0,3386.0,1506.0,1880.0
2,202201,1156011100104450001,11560111,A03,3.0,1,181.83,25.23,156.60,92745.0,...,104.55,50.71,65.170000,45.610000,2.0,0.0,0,3386.0,1506.0,1880.0
3,202201,1156011100104390000,11560111,A04,4.0,1,14.00,3.00,11.00,92745.0,...,104.55,29.72,68.100000,49.390000,2.0,0.0,0,3386.0,1506.0,1880.0
4,202201,1156011100104340000,11560111,A01,5.0,1,295.00,23.00,272.00,92745.0,...,104.55,51.69,68.300000,46.940000,2.0,0.0,0,3386.0,1506.0,1880.0


In [6]:
# train, valid, test로 나누기
train = data[data['train'] == 1]
valid = train[(train['DATA_CRTR_YM'] >= 202211) & (train['DATA_CRTR_YM'] <= 202212)]
train = train[(train['DATA_CRTR_YM'] <= 202210)]
test = data[data['train'] == 0]

In [7]:
train = pd.concat([train.iloc[:, 6:], train['SLS_GRD']], axis = 1)
valid = pd.concat([valid.iloc[:, 6:], valid['SLS_GRD']], axis = 1)
test = test.iloc[:, 6:]
train.head(2)

,전체점포수,프랜차이즈점포수,일반점포수,길단위유동인구,주거인구,직장인구,개업수,폐업수,개업률,폐업률,...,부동산거래대비유동인구,공실률대비매매가임대료,젠트리피케이션,지하철개수,스타벅스개수,대학교여부,65세이상,65세이상_남,65세이상_여,SLS_GRD
0,85.84,1.45,84.39,14929.0,51.0,44.0,3.48,2.03,1.189,0.696,...,19.76,73.540769,51.190526,0.0,0.0,0,2144.0,875.0,1269.0,4.0
1,295.00,23.00,272.00,92745.0,278.0,122.0,10.00,7.00,3.400,2.400,...,51.69,68.320000,43.180000,2.0,0.0,0,3386.0,1506.0,1880.0,5.0


In [8]:
# 전체점포수, 65세이상_남, 65세이상_여 제거
train = train.drop(['전체점포수', '전체 임대료', '65세이상_남', '65세이상_여'], axis = 1)
valid = valid.drop(['전체점포수', '전체 임대료', '65세이상_남', '65세이상_여'], axis = 1)
test = test.drop(['전체점포수', '전체 임대료', '65세이상_남', '65세이상_여'], axis = 1)

In [17]:
print(train.shape)
print(valid.shape)

(961096, 20)
(194944, 20)


In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 961096 entries, 0 to 961095
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   프랜차이즈점포수     961096 non-null  float64
 1   일반점포수        961096 non-null  float64
 2   길단위유동인구      961096 non-null  float64
 3   주거인구         961096 non-null  float64
 4   직장인구         961096 non-null  float64
 5   개업수          961096 non-null  float64
 6   폐업수          961096 non-null  float64
 7   개업률          961096 non-null  float64
 8   폐업률          961096 non-null  float64
 9   1층 임대료       961096 non-null  float64
 10  1층 외 임대료     961096 non-null  float64
 11  생활물가지수       961096 non-null  float64
 12  부동산거래대비유동인구  961096 non-null  float64
 13  공실률대비매매가임대료  961096 non-null  float64
 14  젠트리피케이션      961096 non-null  float64
 15  지하철개수        961096 non-null  float64
 16  스타벅스개수       961096 non-null  float64
 17  대학교여부        961096 non-null  int64  
 18  65세이상        961096 non-

In [23]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194944 entries, 961096 to 1156039
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   프랜차이즈점포수     194944 non-null  float64
 1   일반점포수        194944 non-null  float64
 2   길단위유동인구      194944 non-null  float64
 3   주거인구         194944 non-null  float64
 4   직장인구         194944 non-null  float64
 5   개업수          194944 non-null  float64
 6   폐업수          194944 non-null  float64
 7   개업률          194944 non-null  float64
 8   폐업률          194944 non-null  float64
 9   1층 임대료       194944 non-null  float64
 10  1층 외 임대료     194944 non-null  float64
 11  생활물가지수       194944 non-null  float64
 12  부동산거래대비유동인구  194944 non-null  float64
 13  공실률대비매매가임대료  194944 non-null  float64
 14  젠트리피케이션      194944 non-null  float64
 15  지하철개수        194944 non-null  float64
 16  스타벅스개수       194944 non-null  float64
 17  대학교여부        194944 non-null  int64  
 18  65세이상        19494

In [9]:
X_train, y_train = train.iloc[:, :-1], train.iloc[:, [-1]]
X_valid, y_valid = valid.iloc[:, :-1], valid.iloc[:, [-1]]

In [10]:
y_train = y_train['SLS_GRD'].apply(lambda x: int(x))
y_valid = y_valid['SLS_GRD'].apply(lambda x: int(x))

In [11]:
X_train2 = X_train.drop(['대학교여부'], axis = 1)
X_valid2 = X_valid.drop(['대학교여부'], axis = 1)

In [12]:
features_to_scale = X_train2.columns
scaler = StandardScaler()
scaler.fit(X_train[features_to_scale])
X_train[features_to_scale] = scaler.transform(X_train[features_to_scale])
X_valid[features_to_scale] = scaler.transform(X_valid[features_to_scale])

In [13]:
X_train

,프랜차이즈점포수,일반점포수,길단위유동인구,주거인구,직장인구,개업수,폐업수,개업률,폐업률,1층 임대료,1층 외 임대료,생활물가지수,부동산거래대비유동인구,공실률대비매매가임대료,젠트리피케이션,지하철개수,스타벅스개수,대학교여부,65세이상
0,-0.708988,-0.325084,-1.258521,-1.359992,-0.472021,-0.242928,-0.464173,-0.461519,-0.603607,0.562311,-0.593268,-1.863554,-2.200829,0.569375,0.243689,-1.032688,-0.098563,0,-1.077794
1,0.086300,0.771074,0.722326,0.592735,-0.065615,0.723879,0.218004,0.244111,-0.102444,-0.775482,-0.760337,-1.863554,0.214040,-0.214749,-0.937152,0.833506,-0.098563,0,-0.319485
2,0.168597,0.096821,0.722326,0.592735,-0.065615,0.359102,-0.145732,-0.461519,-0.603607,-0.775482,-0.760337,-1.863554,0.139923,-0.687858,-0.578943,0.833506,-0.098563,0,-0.319485
3,-0.651786,-0.753883,0.722326,0.592735,-0.065615,-0.758954,-0.605550,-0.840982,1.279872,-0.775482,-0.760337,-1.863554,-1.447554,-0.247792,-0.021729,0.833506,-0.098563,0,-0.319485
4,0.086300,0.771074,0.722326,0.592735,-0.065615,0.723879,0.218004,0.244111,-0.102444,-0.775482,-0.760337,-1.863554,0.214040,-0.217753,-0.382886,0.833506,-0.098563,0,-0.319485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961091,-0.651786,-0.601972,-1.149419,-0.508362,-0.617910,-0.758954,-0.742809,-0.840982,-0.808307,-1.410434,-0.917811,1.214615,1.889247,-1.356915,-0.087521,-1.032688,-0.098563,0,0.868043
961092,-0.651786,-0.806468,-1.149419,-0.508362,-0.617910,-0.758954,-0.742809,-0.840982,-0.808307,-1.410434,-0.917811,1.214615,2.164541,-1.356915,-0.087521,-1.032688,-0.098563,0,0.868043
961093,-0.651786,-0.356577,-0.897562,-0.215883,-0.654383,-0.165821,0.080745,0.722828,1.338694,-1.312876,-0.952965,1.214615,1.943701,-0.339410,0.110941,-1.032688,-0.098563,0,0.395474
961094,-0.762499,-0.701299,-0.897562,-0.215883,-0.654383,-0.610670,-0.742809,0.754743,-0.808307,-1.312876,-0.952965,1.214615,0.076393,-1.908929,-0.857550,-1.032688,-0.098563,0,0.395474


In [14]:
model = RandomForestClassifier(random_state = 0)

In [15]:
model.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [21]:
pred = model.predict(X_valid)
accuracy = accuracy_score(y_valid, pred)
print('랜덤 포레스트 정확도: {:.4f}'.format(accuracy))

랜덤 포레스트 정확도: 0.6360
